# BreastMammaryTissueJunctionAnalysis as a Notebook 

rMATS 3.2.5 was run on controlled access RNASeq files retrieved experiments stored in the Sequence Read Archive with controlled access managed by dbGaP.   The data were generated under the Gene Tissue Expression.

## rMATS RNASeq-MATS.py produces 10 different output types which get assembled into as type junction ID by sample ID matrices

### Alternative Splice Site Types are: (se, a3ss, a5ss, mxe, ri)

 This is input as ARGV1 into variable 'astype'

  * Skipped Exon events (se),
  * Alternative 3' splice site (a3ss),
  * Alternative 5' splice site (a5ss),
  * Mutually exclusive exon (mxe),
  * and retention intron (ri)

### There are two different kinds of junction counts

  * jc = junction counts - reads that cross the junction
  * jcec = junction counts plus reads on the target (such as included exon

### And the count type -- there are 5 types

  * inclusion levels (percent spliced in)
  * included junction counts (ijc)
  * skipped junction counts (sjc)
  * inclusion length (inclen)
  * skipped length (skiplen)

### function: fit_iso_tissue 

fit_iso_tissue expects the following input:

  * the tissue of interest (SMSTD) 
  * an ordered_merged_rmats -- which will be ordered to fit the count matrix
  * count matrix (inc or ijc & sjc merged)
  * splice type (a3ss, a5ss, mxe, ri or se)
  * junction_count type (jc or jcec)
  * count type (inc or the merged ijc,sjc)
  
### reordering to match annotations between count matrix and annotation matrix

Common problem is to match specifically the rows of an annotation matrix with the columns of a count matrix
`match` is the function that gives the re-ordering index required to accomplish this


## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## rMATS-final-merged
the rmats-nf NextFlow was executed and the results released here:

## Loading dependencies

In [1]:
library(limma)
library(piggyback)
library(multtest)
library(Biobase)
library(edgeR)
library(tidyverse)
library(R.utils)
library(statmod)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:limma’:

    plotMA

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: Biobase
Welcome to Bio

## Modeling

This analysis uses edgeR.  From the documentation, it is important to note that normalization takes the form of correction factors that enter into the statistical model. Such correction factors are usually computed internally by edgeR functions, but it is also possible for a user to supply them. The correction factors may take the form of scaling factors for the library sizes, such as computed by calcNormFactors, which are then used to compute the effective library sizes. 

Alternatively, gene-specific correction factors can be entered into the glm functions of edgeR as offsets. In the latter case, the offset matrix will be assumed to account for all normalization issues, including sequencing depth and RNA composition.

Note that normalization in edgeR is model-based, and the original read counts are not themselves transformed. This means that users should not transform the read counts in any way before inputing them to edgeR. For example, users should not enter RPKM or FPKM val- ues to edgeR in place of read counts. Such quantities will prevent edgeR from correctly estimating the mean-variance relationship in the data, which is a crucial to the statistical strategies underlying edgeR. Similarly, users should not add artificial values to the counts before inputing them to edgeR.

edgeR is not designed to work with estimated expression levels, for example as might be output by Cufflinks. 
edgeR can work with expected counts as output by RSEM, but raw counts are still preferred. 

As instructed by the software, we are using the raw counts as provided by rMATS.  The raw counts we are using in the model are `ijc` and `sjc`, the sample specific raw read counts as they align to the junctions of the `included exon (ijc)` and the junctions of the `excluded or skipped exon (sjc)` respectively.


Be sure to set your GITHUB_TOKEN, prior to downloading files

One suggestion is change it to your token and then run it then immediately change it back to this:

Sys.setenv(GITHUB_TOKEN = "your-very-own-github-token")

### Did you remember?
Did you remember to delete your private github token?  Now is a good time to do so, before you save your work and checkit in inadvertantly....

In [ ]:
piggyback::pb_download(
    repo = "TheJacksonLaboratory/sbas", 
    file = "SraRunTable.noCram.noExome.noWGS.totalRNA.txt.gz",
    tag  = "GTExV8.v1.0", 
    dest = "../data/")

piggyback::pb_download(
    repo = "adeslatt/sbas_test", 
    file = "rmats_final.se.jc.ijc.txt.gz",
    tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
    dest = "../data/")

piggyback::pb_download(
    repo = "adeslatt/sbas_test", 
    file = "rmats_final.se.jc.sjc.txt.gz",
    tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
    dest = "../data/")


In [6]:

ijc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.ijc.txt.gz") 
sjc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.sjc.txt.gz") 
meta.data          <- data.table::fread("../data/SraRunTable.txt.gz")

head(ijc.iso.counts.mem)
head(sjc.iso.counts.mem)
head(meta.data)

#dimensions before we make the changes.
dim(ijc.iso.counts.mem)
dim(sjc.iso.counts.mem)
dim(meta.data)

ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880,SRR1068929,SRR1068953,SRR1068977,SRR1068999,⋯,SRR821573,SRR821581,SRR821602,SRR821626,SRR821653,SRR821690,SRR821715,SRR823967,SRR823991,SRR824015
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,26,247,103,620,494,145,145,139,697,⋯,151,32,62,48,963,25,196,76,72,61
3,1,0,1,0,0,0,1,1,2,⋯,2,1,0,1,3,0,1,0,0,0
4,0,1,1,2,0,0,1,0,2,⋯,0,0,0,0,1,0,0,0,0,0
5,3,0,2,3,6,1,1,1,5,⋯,3,2,0,1,6,0,2,0,0,0
6,2,1,2,5,6,1,1,0,5,⋯,1,1,0,0,4,0,1,0,0,0


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880,SRR1068929,SRR1068953,SRR1068977,SRR1068999,⋯,SRR821573,SRR821581,SRR821602,SRR821626,SRR821653,SRR821690,SRR821715,SRR823967,SRR823991,SRR824015
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,2,0,1,3,6,1,0,0,3,⋯,1,1,0,0,3,0,1,0,0,0
2,0,0,0,1,0,0,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,⋯,0,0,0,0,2,0,1,0,0,0
5,0,5,3,8,4,0,3,0,3,⋯,9,3,1,2,3,0,1,0,1,0
6,11,119,36,284,207,60,63,43,295,⋯,52,13,14,9,338,8,63,25,20,18


Run,analyte_type,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,biospecimen_repository,biospecimen_repository_sample_id,body_site,⋯,data_type (run),product_part_number (exp),product_part_number (run),sample_barcode (exp),sample_barcode (run),is_technical_control,target_set (exp),primary_disease (exp),secondary_accessions (run),Alignment_Provider (run)
<chr>,<chr>,<chr>,<int>,<int64>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
SRR3136805,DNA,WGS,302,116341565242,PRJNA244100,SAMN02649364,Cloud Testing,NA12878,Peripheral blood,⋯,,,,NA,NA,,,,,IRC Harmonized
SRR1219802,DNA,WGS,201,25638643152,PRJNA244100,SAMN02652271,Cloud Testing,NA19020,Peripheral blood,⋯,,,,NA,NA,,,,,
SRR1219803,DNA,WGS,201,23409785454,PRJNA244100,SAMN02652298,Cloud Testing,NA19027,Peripheral blood,⋯,,,,NA,NA,,,,,
SRR1219804,DNA,WGS,201,23806900587,PRJNA244100,SAMN02652211,Cloud Testing,NA19028,Peripheral blood,⋯,,,,NA,NA,,,,,
SRR1219805,DNA,WGS,201,30424860846,PRJNA244100,SAMN02652247,Cloud Testing,NA19031,Peripheral blood,⋯,,,,NA,NA,,,,,
SRR1219806,DNA,WGS,201,23992100045,PRJNA244100,SAMN02652244,Cloud Testing,NA19035,Peripheral blood,⋯,,,,NA,NA,,,,,


[1] 42611  8674

[1] 42611  8674

[1] 24667    79

## Synchronize metadata samples with ijc sjc samples

Keep only the runs that are in the ijc count list (assuming ijc and sjc are the same).  As well, name the rows with the junction id column and then make the matrix just about the counts.

In [ ]:
# the sample names are in the columns of both the ijc and the sjc matrices, these matrices have the identical column order)
keep.meta.data <- meta.data$Run %in% colnames(ijc.iso.counts.mem)
table(keep.meta.data)
reduced.meta.data <- meta.data[keep.meta.data==TRUE,]

# preserve junction id as rowname
rownames(ijc.iso.counts.mem) <- ijc.iso.counts.mem$ID
rownames(sjc.iso.counts.mem) <- sjc.iso.counts.mem$ID

# and remove the id to have a data matrix
ijc.iso.counts.mem  <- ijc.iso.counts.mem[,-1]
sjc.iso.counts.mem  <- sjc.iso.counts.mem[,-1]

meta.data.run.names  <- reduced.meta.data$Run
ijc.iso.counts.mem2  <- as_tibble(ijc.iso.counts.mem)
sjc.iso.counts.mem2  <- as_tibble(sjc.iso.counts.mem)

ijc.iso.counts.mem2  <- ijc.iso.counts.mem2[,c(meta.data.run.names)]
sjc.iso.counts.mem2  <- sjc.iso.counts.mem2[,c(meta.data.run.names)]

dim(ijc.iso.counts.mem)
dim(sjc.iso.counts.mem)
dim(reduced.meta.data)

## Order ijc and sjc columns in the same order as the metadata Run order

Using tibble library, we can rearrange the columns as the column name.  

In [ ]:
meta.data.run.names  <- as.character(reduced.meta.data$Run)
ijc.iso.counts.mem2  <- as_tibble(ijc.iso.counts.mem)
sjc.iso.counts.mem2  <- as_tibble(sjc.iso.counts.mem)

ijc.iso.counts.mem2  <- ijc.iso.counts.mem2[,c(meta.data.run.names)]
sjc.iso.counts.mem2  <- sjc.iso.counts.mem2[,c(meta.data.run.names)]

dim(ijc.iso.counts.mem2)
dim(sjc.iso.counts.mem2)
dim(reduced.meta.data)

Remove samples that match '11IL0' from the ijc, sjc and metadata files using the logical grep, grepl

In [ ]:
keep.meta.data <- (!grepl('11ILO',reduced.meta.data$"Sample Name"))
table(keep.meta.data)
ijc.iso.counts.mem2 <-ijc.iso.counts.mem2 [                    ,keep.meta.data==TRUE]
sjc.iso.counts.mem2 <-sjc.iso.counts.mem2 [                    ,keep.meta.data==TRUE]

reduced.meta.data   <-reduced.meta.data   [keep.meta.data==TRUE,                    ]
dim(ijc.iso.counts.mem2)
dim(sjc.iso.counts.mem2)

### and focus on a single tissue

this will become a function so we can proceed on all the tissues

In [ ]:
tissue <- reduced.meta.data$body_site %in% 'Breast - Mammary Tissue'
table(tissue)

ijc.iso.counts.mem2 <-ijc.iso.counts.mem2 [                    ,tissue==TRUE]
sjc.iso.counts.mem2 <-sjc.iso.counts.mem2 [                    ,tissue==TRUE]

reduced.meta.data   <-reduced.meta.data   [tissue==TRUE,                    ]
dim(ijc.iso.counts.mem2)
dim(sjc.iso.counts.mem2)

### exploration of the details

For each sample, we have ijc and sjc count data and demographics of gender.
Our question is regarding the sex biased differences.
For each junction we have 8,000 samples with these count data.   The way to think about the model is that we have in fact for all of these junctions, these are our co-variates in this global transcriptomic model. 
For exon skipping events (SE), we have 42,611 non-zero junction IDs the (first dimension of the ijc and sjc cout table) for the skipped exon event for breast-Mammary Tissue, 191 individuals.  These are healthy individuals, and we are studying the impact of sex on the occurrence or non-occurance of specific alternative splicing events.   We explore the information we ahve about these junctions and create a construct, as_event, which accounts for the junction under exploration.

In [ ]:
ijc      <- as.data.frame(ijc.iso.counts.mem2)
sjc      <- as.data.frame(sjc.iso.counts.mem2)
ijc      <- data.matrix(ijc)
sjc      <- data.matrix(sjc)

## Exploring the ijc and sjc Count data 

We have two counts that are in many ways two sides of the same coin.  Both our the observational output and we wish to see how robust each are in their ability to separate out the samples to provide for us differentially expressed isoform events as measured by their counts.   Each junction is in a manner a specific marker to specific isoform events that may or may not be shared between the genders.   If there is significant results, then this is indicative of the separation achieved by isoform specific differentiation.   In our model we will use these in combination, it is important to see if they will yield the results we are looking for.

### IJC

Exon included junction counts -- duplicate correlation is actually 2 (there are 2 exons for each included exon) the results provide robust separation up until around 550.

In [ ]:
donor <- factor(substr(reduced.meta.data$'Sample Name', 1,9))
length(levels(donor))

In [ ]:
sex       <- factor(reduced.meta.data$sex,levels=c('male','female'))

design    <- model.matrix ( ~ sex + donor)
y <- DGEList(counts=ijc, group = sex)
y <- calcNormFactors(y, method="upperquartile")
y_voom <- voom (y, design=design)

Gender <- substring(sex,1,1)
plotMDS(y, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y, labels=Gender, top=1500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom, labels=Gender, top=1500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

### SJC
sjc counts are skipped exon junction counts -- ijc counts hold together for the top 1000, the skipped exon junction counts fall appart
at a lower number -- around 200!  separation fails between the genders at 250

In [ ]:
sex       <- factor(reduced.meta.data$sex,levels=c('male','female'))
design    <- model.matrix(~ sex)
y <- DGEList(counts=sjc, group = sex)
y <- calcNormFactors(y, method="upperquartile")
y_voom <- voom (y, design=design)

Gender <- substring(sex,1,1)
plotMDS(y, labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom, labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y, labels=Gender, top=700, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom, labels=Gender, top=700, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

## Differential analysis as_event:ijc 

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + epsilon (error)
           

where y is the included exon junction count expression; sex denotes the reported sex of the subject.

In [ ]:
sex       <- reduced.meta.data$sex

design    <- model.matrix( ~ sex )

colnames(design) <- c("intercept","sex")

dim(ijc)
table(sex)
head(design)

y_ijc <- DGEList(counts=ijc, group = sex)
y_ijc <- calcNormFactors(y_ijc, method="upperquartile")

y_ijc_voom <- voom (y_ijc, design=design)

Gender <- substring(sex,1,1)

plotMDS(y_ijc, labels=Gender, top=1000, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

plotMDS(y_ijc_voom, labels=Gender, top=1000, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

fit_ijc <- lmFit(y_ijc_voom, design)
fit_ijc <- eBayes(fit_ijc)

ijc_sex_results          <- topTable(fit_ijc, coef='sex', number=nrow(y_voom))
ijc_sex_results_refined  <- ijc_sex_results$adj.P.Val < 0.05 & abs(ijc_sex_results$logFC) > 1.5

table(ijc_sex_results_refined)

## Differential analysis as_event:sjc 

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + epsilon (error)
           

where y is the excluded exon junction count (sjc) expression; sex denotes the reported sex of the subject.

In [ ]:
sex       <- reduced.meta.data$sex

design    <- model.matrix( ~ sex )

colnames(design) <- c("intercept","sex")

dim(sjc)
table(sex)
head(design)

y_sjc <- DGEList(counts=sjc, group = sex)
y_sjc <- calcNormFactors(y_sjc, method="upperquartile")

y_sjc_voom <- voom (y_sjc, design=design)

Gender <- substring(sex,1,1)

plotMDS(y_sjc, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

plotMDS(y_sjc_voom, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

fit_sjc <- lmFit(y_sjc_voom, design)
fit_sjc <- eBayes(fit_sjc)

sjc_sex_results          <- topTable(fit_sjc, coef='sex', number=nrow(y_voom))
sjc_sex_results_refined  <- sjc_sex_results$adj.P.Val < 0.05 & abs(sjc_sex_results$logFC) > 1.5

table(sjc_sex_results_refined)

## Differential analysis as_event (combined ijc and sjc)

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + B2 as_event + B3 sex*as_event + epsilon (error)
           

where y is the alternative splicing event expression; sex denotes the reported sex of the subject, as_event represents the specific alternative splicing event - either included exon junction counts or skipped exon junction counts and their interaction terms.   Donor is added to our model as a blocking variable used in both the calculation of duplicate correlation as well as in the linear fit.

In [ ]:
ijc_names    <- as.character(colnames(ijc))
sjc_names    <- as.character(colnames(sjc))
sample_names <- as.character(colnames(ijc))

ijc_names <- paste0(ijc_names,"-ijc")
sjc_names <- paste0(sjc_names,"-sjc")
colnames(ijc) <- ijc_names
colnames(sjc) <- sjc_names

as_matrix <- cbind(ijc,sjc)
sex       <- c(rep(reduced.meta.data$sex,2))
sex       <- factor(sex, levels=c('male','female'))
as_event  <- c(rep("ijc",dim(ijc)[2]), rep("sjc", dim(sjc)[2]))
as_event  <- factor(as_event, levels=c("ijc", "sjc"))

# we will add donor as a blocking parameter
donor    <- rep(sample_names, 2)

design    <- model.matrix( ~ sex + as_event + sex*as_event )

colnames(design) <- c("intercept","sex","as_event","sex*as_event")

dim(as_matrix)
table(sex)
table(as_event)
head(design)

### Voom, limma's lmFit and eBayes

Using sample as a blocking variable, we are able to model the effects of the donor on the results, which improves the power.  This topic is discussed in biostars https://www.biostars.org/p/54565/.  And Gordon Smyth answers the question here https://mailman.stat.ethz.ch/pipermail/bioconductor/2014-February/057887.html.  The method of modeling is a random effects approach in which the intra-donor correlation is incorporated into the covariance matrix instead of the linear predictor.   And though as Gordon Smyth states both are good method and the twoway anova approach makes fewer assumptions, the random effects approach is statistically more powerful.  

We have a balanced design in which all donors receive all stimuli (which is really in healthy human donors, life and all of its factors!) Our measurement has so many points -- we are measuring in the skipped exon approach, 42,611 junctions!   It is not possible to encorporate those measurements into the linear predictor.  A two-way ANOVA approach is virtually as powerful as the random effects approach 
and hence is preferable as it makes fewer assumptions.

For an unbalanced design in which each donor receives only a subset of the stimula, the random effects approach is more powerful.

Random effects approach is equivalent to The first method is twoway anova, a generalization of a paired analysis.


In [ ]:
# we will model as random effects, represented as a block
donor2    <- rep(donor, 2)
length(levels(donor2))


In [ ]:
y <- DGEList(counts=as_matrix, group = sex)
y <- calcNormFactors(y, method="upperquartile")
y_voom <- voom (y, design=design)

dup_cor <- duplicateCorrelation(y_voom$E, design=design, ndups=2, block=donor, weights=y$samples$norm.factors)
dup_cor$consensus.correlation

In [ ]:
y_dup_voom <- voom (y, design=design, plot = TRUE, block = donor, correlation = dup_cor$consensus.correlation)

In [ ]:
Gender <- substring(sex[1:dim(ijc)[2]],1,1)

plotMDS(y[,c(1:dim(ijc)[2])], labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom[,c(1:dim(ijc)[2])], labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_dup_voom[,c(1:dim(ijc)[2])], labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y[,c((dim(ijc)[2]+1)):(dim(ijc)[2]+dim(sjc)[2])], labels=Gender, top=250, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom[,c((dim(ijc)[2]+1)):(dim(ijc)[2]+dim(sjc)[2])], labels=Gender, top=250, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_dup_voom[,c((dim(ijc)[2]+1):(dim(ijc)[2]+dim(sjc)[2]))], labels=Gender, top=250, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

In [ ]:
fit <- lmFit(y_dup_voom, design=design, block=donor, correlation = dup_cor$consensus.correlation)
fit <- eBayes(fit, robust=TRUE)

In [ ]:
sex_as_events_results         <- topTable(fit, coef="sex*as_event", number=nrow(y_voom))
sex_as_events_results_refined <- sex_as_events_results$adj.P.Val < 0.05 & abs(sex_as_events_results$logFC) > 1.5

sex_results                   <- topTable(fit, coef="sex", number=nrow(y_voom))
sex_results_refined           <- sex_results$adj.P.Val < 0.05 & abs(sex_results$logFC) > 1.5

table(sex_as_events_results_refined)
table(sex_results_refined)

#### fromGTF

rMATS analyzes the gtf file and determines the junctions, identifying 10 matrices in all.  The fromGTF.SE.txt contains the annotation information and we can use this information to estimate the duplicate correlation effect from counting based upon the same gene with our ijc counts.  we will use this information, to obtain the gene information.


In [ ]:
# fromGTF.tar.gz
if (! (file.exists("../data/fromGTF.tar.gz"))) {
    system("mkdir -p ../data", intern = TRUE)
    message("Fetching fromGTF.tar.gz from GitHub ..")
    # Download archive from GitHub release with tag "dge"
    piggyback::pb_download(file = "fromGTF.tar.gz",
                           dest = "../data",
                           repo = "adeslatt/sbas_gtf",
                           tag  = "rMATS.3.2.5.gencode.v30",
                           show_progress = TRUE)
    message("Done!\n")
    message("Decompressing fromGTF.tar.gz into ../data")
    system("mkdir -p ../data && tar xvfz ../data/fromGTF.tar.gz -C ../data", intern = TRUE)
    message("Done!\n")
    message("Decompressing fromGTF.*.txt.gz into ../data")
    system("gunzip  ../data/fromGTF*.txt.gz ", intern = TRUE)
    message("Done!\n")
}
fromGTF.SE <- read.table("../data/fromGTF.SE.txt", header=TRUE)
head(fromGTF.SE)
genes <- factor(fromGTF.SE$geneSymbol)
length(levels(genes))

In [ ]:
head(sex_as_events_results)

In [ ]:
# index to the annotations - these are the Junction IDs
sex_as_events_rnResults <- rownames(sex_as_events_results)
sex_rnResults           <- rownames(sex_results)
ijc_sex_rnResults       <- rownames(ijc_sex_results)
sjc_sex_rnResults       <- rownames(sjc_sex_results)
head(sex_as_events_rnResults)
head(ijc_sex_rnResults)
head(sjc_sex_rnResults)
head(sex_rnResults)
head(fromGTF.SE[sex_as_events_rnResults,])

In [ ]:
# use the junctionIDs to get the annotations
sex_as_events_resultsAnnotations      <- fromGTF.SE[sex_as_events_rnResults,]
sex_resultsAnnotations                <- fromGTF.SE[sex_rnResults,]
ijc_sex_resultsAnnotations            <- fromGTF.SE[ijc_sex_rnResults,]
sjc_sex_resultsAnnotations            <- fromGTF.SE[sjc_sex_rnResults,]
head(sex_as_events_resultsAnnotations)
head(sex_resultsAnnotations)
head(ijc_sex_resultsAnnotations)
head(sjc_sex_resultsAnnotations)

In [ ]:
sex_as_events_results_refinedAnnotations<- sex_as_events_resultsAnnotations[sex_as_events_results_refined==TRUE,]
sex_results_refinedAnnotations          <- sex_resultsAnnotations          [sex_results_refined          ==TRUE,]
ijc_sex_results_refinedAnnotations      <- ijc_sex_resultsAnnotations      [ijc_sex_results_refined      ==TRUE,]
sjc_sex_results_refinedAnnotations      <- sjc_sex_resultsAnnotations      [sjc_sex_results_refined      ==TRUE,]
head(sex_as_events_results_refinedAnnotations)
head(sex_results_refinedAnnotations)
head(ijc_sex_results_refinedAnnotations)
head(sjc_sex_results_refinedAnnotations)

In [ ]:

# geneSymbols are in the annotations
sex_as_events_geneSymbols         <- sex_as_events_resultsAnnotations$geneSymbol
sex_geneSymbols                   <- sex_resultsAnnotations$geneSymbol
ijc_sex_geneSymbols               <- ijc_sex_resultsAnnotations$geneSymbol
sjc_sex_geneSymbols               <- sjc_sex_resultsAnnotations$geneSymbol

sex_as_events_refined_geneSymbols <- sex_as_events_results_refinedAnnotations$geneSymbol
sex_refined_geneSymbols           <- sex_results_refinedAnnotations$geneSymbol
ijc_sex_refined_geneSymbols       <- ijc_sex_results_refinedAnnotations$geneSymbol
sjc_sex_refined_geneSymbols       <- sjc_sex_results_refinedAnnotations$geneSymbol

# adjust the rownames to be the geneSymbols rather than junction IDs
sex_as_events_results_rn   <- paste(sex_as_events_geneSymbols, sex_as_events_rnResults, sep="-")
sex_results_rn             <- paste(sex_geneSymbols,           sex_rnResults, sep="-")
ijc_sex_results_rn         <- paste(ijc_sex_geneSymbols,       ijc_sex_rnResults, sep="-")
sjc_sex_results_rn         <- paste(sjc_sex_geneSymbols,       sjc_sex_rnResults, sep="-")

head(sex_as_events_results_rn)
head(sex_results_rn)
head(ijc_sex_results_rn)
head(sjc_sex_results_rn)


In [ ]:
rownames(sex_as_events_results) <- sex_as_events_results_rn
rownames(sex_results)           <- sex_results_rn
rownames(ijc_sex_results)       <- ijc_sex_results_rn
rownames(sjc_sex_results)       <- sjc_sex_results_rn

In [ ]:
sex_as_events_filename         = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sex_as_events.csv',sep='')
sex_filename                   = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sex.csv',sep='')
ijc_sex_filename               = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_ijc_sex.csv',sep='')
sjc_sex_filename               = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sjc_sex.csv',sep='')

sex_as_events_refined_filename = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sex_as_events_refined.csv',sep='')
sex_refined_filename           = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sex_refined.csv',sep='')
ijc_sex_refined_filename       = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_ijc_sex_refined.csv',sep='')
sjc_sex_refined_filename       = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sjc_sex_refined.csv',sep='')

sex_as_events_genesFilename    = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sex_as_events_universe.txt',sep='')
sex_genesFilename              = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sex_universe.txt',sep='')
ijc_sex_genesFilename          = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_ijc_sex_universe.txt',sep='')
sjc_sex_genesFilename          = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sjc_sex_universe.txt',sep='')

sex_as_events_refined_genesFilename = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sex_as_events_gene_set.txt',sep='')
sex_refined_genesFilename           = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sex_gene_set.txt',sep='')
ijc_sex_refined_genesFilename       = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_ijc_sex_gene_set.txt',sep='')
sjc_sex_refined_genesFilename       = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sjc_sex_gene_set.txt',sep='')


In [ ]:
write.table(sex_as_events_results, file = sex_as_events_filename, row.names = T, col.names = T, quote = F, sep = ",")
write.table(sex_results,           file = sex_filename          , row.names = T, col.names = T, quote = F, sep = ",")
write.table(ijc_sex_results,       file = ijc_sex_filename      , row.names = T, col.names = T, quote = F, sep = ",")
write.table(sjc_sex_results,       file = sjc_sex_filename      , row.names = T, col.names = T, quote = F, sep = ",")

write.table(sex_as_events_results[sex_as_events_results_refined,], file = sex_as_events_refined_filename, row.names = T, col.names = T, quote = F, sep = ",")
write.table(sex_results          [sex_results_refined          ,], file = sex_refined_filename          , row.names = T, col.names = T, quote = F, sep = ",")
write.table(ijc_sex_results      [ijc_sex_results_refined      ,], file = ijc_sex_refined_filename      , row.names = T, col.names = T, quote = F, sep = ",")
write.table(sjc_sex_results      [sjc_sex_results_refined      ,], file = sjc_sex_refined_filename      , row.names = T, col.names = T, quote = F, sep = ",")

write.table(sex_as_events_geneSymbols, file = sex_as_events_genesFilename, row.names = F, col.names = F, quote = F, sep = ",")
write.table(sex_geneSymbols,           file = sex_genesFilename          , row.names = F, col.names = F, quote = F, sep = ",")
write.table(ijc_sex_geneSymbols,       file = ijc_sex_genesFilename      , row.names = F, col.names = F, quote = F, sep = ",")
write.table(sjc_sex_geneSymbols,       file = sjc_sex_genesFilename      , row.names = F, col.names = F, quote = F, sep = ",")

write.table(sex_as_events_refined_geneSymbols,file = sex_as_events_refined_genesFilename, row.names = F, col.names = F, quote = F, sep = ",")
write.table(sex_refined_geneSymbols,          file = sex_refined_genesFilename          , row.names = F, col.names = F, quote = F, sep = ",")
write.table(ijc_sex_refined_geneSymbols,      file = ijc_sex_refined_genesFilename      , row.names = F, col.names = F, quote = F, sep = ",")
write.table(sjc_sex_refined_geneSymbols,      file = sjc_sex_refined_genesFilename      , row.names = F, col.names = F, quote = F, sep = ",")


## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [ ]:
notebookid   = "BreastMammaryTissueJunctionAnalysis"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data && find . -type f -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", notebookid, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", notebookid, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", notebookid ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]